# ЧТО ДАЛЬШЕ?

- Как определить, что новая рекомендательная система стала лучше?
- А/Б-тестирование.
- Однорукие бандиты.
- Элементы Reinforcement Learning.

**МЕТРИКИ КАЧЕСТВА**

**Prediction Accuracy** — оценивают точность предсказываемого рейтинга.
<img src="../images/ml9_24.png" alt="Binary-cross-entropy" width="700" align="center">

**Decision support** — оценивают релевантность рекомендаций.

<img src="../images/ml9_25.png" alt="Binary-cross-entropy" width="700" align="center">

**Rank Accuracy метрики** — оценивают качество ранжирования выдаваемых рекомендаций.

<img src="../images/ml9_26.png" alt="Binary-cross-entropy" width="700" align="center">

# КАКИЕ ГРАНИ РЕКОМЕНДАТЕЛЬНЫХ СИСТЕМ СЕГОДНЯ МОЖНО СОВЕРШЕНСТВОВАТЬ?

- Принимать во внимание контекст рекомендации (где находится пользователь, что он делает/делал ранее).
- Может оцениваться несколько характеристик товара, в этом случае должна учитываться каждая в отдельности.
- Иметь возможность работать не с пользователями, а с группами пользователей (всей группе рекомендуем товар X).
- При показе рекомендаций учитывать различные бизнес-правила, фильтры и ограничения.
- Фокусироваться на ранжировании рекомендаций, а не на оценке конкретного рейтинга.
- Обращать внимание на accuracy/novely tradeoff (часто интереснее советовать что-то принципиально новое).
- Вместо близости предпочтений учитывать социальную близость между пользователями (друзья в Facebook).
- Система должна подстраиваться под пользователя, пользователь должен иметь возможность настраивать систему.
- Прозрачность рекомендаций (важно понимать, почему показана именно такая).
- Развивать устойчивость к искусственным манипуляциям (чтобы фейковые отзывы не влияли на оценку).

# Выводы

- Классические методы рекомендаций позволяют получить быстрый результат.
- Нейронные сети позволяют строить более гибкие рекомендации, учитывая последовательность покупок.
- Word2vec и ассоциативные правила позволяют визуализировать результат.

# Задачи

Для решения заданий будут необходимы следующие пакеты:

In [2]:
import pandas as pd
import numpy as np
import surprise
import apyori

Библиотека surprise — это специализированная библиотека для работы с рекомендательными системами, более подробно можно познакомиться с ней здесь http://surpriselib.com 

В данной библиотеке есть встроенные датасеты, одним из которых мы и будем пользоваться. Загрузим датасет ml-100k:

In [70]:
data = surprise.Dataset.load_builtin("ml-100k")
trainingSet = data.build_full_trainset()

Важно! Если при запуске предыдущей ячейки вам выходит сообщение о предложении скачать датасет, ответьте на него утвердительно.

In [4]:
testSet = trainingSet.build_testset()

Задание 9.10.1

Найдите количество пользователей в trainingSet.

In [15]:
trainingSet.n_users

943

Задание 9.10.2

Найдите количество items в trainingSet.

In [17]:
trainingSet.n_items

1682

Задание 9.10.3

Выведите среднее значение по всем показателям рейтинга для trainingSet. Ответ округлите до сотых.


In [19]:
trainingSet.global_mean

3.52986

Задание 9.10.4

Проведите тренировку алгоритма user-based коллаборативной фильтрации KNNBasic.

Постройте рекомендации testset c использованием этого алгоритма. Выведите предсказание под  8-ым индексом. В качестве ответа укажите id рекомендованного фильма.

In [41]:
from surprise import KNNBasic

# инициализирууем модель(алгоритм) и обучим его
model = KNNBasic()
model.fit(trainingSet)

# получим предсказания
pred_knnb = model.test(testSet)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [42]:
pred_knnb[8].iid

'663'

Задание 9.10.5

Проведите тренировку алгоритма SVD. Укажите параметр random_state=829. Постройте рекомендации testset c использованием этого алгоритма. Выведите предсказание под 42-ым индексом. 

В качестве ответа укажите разницу между истинным значением рейтинга для этого предсказания и оцениваемым. Округлите до тысячных.

In [43]:
from surprise import SVD

# инициализирууем модель(алгоритм) и обучим его
model = SVD(random_state=829)
model.fit(trainingSet)

# получим предсказания
pred_svd = model.test(testSet)

In [44]:
pred_svd[42]

Prediction(uid='186', iid='148', r_ui=4.0, est=3.575883908398154, details={'was_impossible': False})

In [45]:
round(abs(pred_svd[42].r_ui - pred_svd[42].est), 3)

0.424

Задание 9.10.6

Рассчитайте метрику качества RMSE для предсказаний через функцию surprise.accuracy.rmse() для обоих обученных ранее алгоритмов. 

В качестве ответа укажите значение RMSE для того алгоритма, который дал лучший результат. Округлите ответ до двух знаков после точки-разделителя.

In [46]:
from surprise import accuracy

# compute RMSE
rmse_knnb = accuracy.rmse(pred_knnb)
rmse_svd  = accuracy.rmse(pred_svd)

print(f'rmse_knnb = {rmse_knnb}, rmse_svd = {rmse_svd}')

RMSE: 0.7781
RMSE: 0.6747
rmse_knnb = 0.7780707712292397, rmse_svd = 0.6746987152322735


Задание 9.10.7

Cформируем датафрейм из нашего датасета.

In [71]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 3]
df.head()

,user_id,movie_id,rating,3
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


1. Сформируйте для каждого user_id текстовую строку, содержащую набор movie_id, получивших оценку 4 и выше. Используйте пробел (" ") в качестве разделителя.

Для проверки результата введите строку для user_id 100. Укажите параметр random_state=829.

In [78]:
good = df[df['rating'] >= 4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(a) for a in r]))
good

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
                             ...                        
95     625 233 68 1 527 172 1206 416 174 143 96 843 1...
96     156 87 673 479 153 91 519 8 484 645 64 100 170...
97     228 222 670 89 482 98 50 115 7 435 28 428 655 ...
98     47 517 116 629 523 514 25 428 435 211 210 659 194
99     4 79 1016 403 50 742 181 182 597 410 168 204 5...
Name: movie_id, Length: 942, dtype: object

In [79]:
good['100']

'344 355 750 302 691 316 752 313 879 300 328 1235 690 347 269 258 900 294 272 751 898 315'

In [77]:
df.query(f'user_id == "100" and rating == 5')

,user_id,movie_id,rating,3
8793,100,316,5.0,891375313
18058,100,313,5.0,891374706
97602,100,315,5.0,891375557


2. Используя алгоритм apriory из модуля apyory, рассчитайте ассоциативные правила для полученного набора фильмов с положительными оценками. Будем считать, что хороший рейтинг — это когда оценка >=4. Используйте следующие ограничения:

  - минимальный support 0.2,
  - минимальное значение confidence 0.3,
  - минимальное значение lift 2,
  - минимальная длина 2.

Сколько правил получилось?

In [56]:
good = df[df['rating'] >= 4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(a) for a in r]))
good

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
                             ...                        
95     625 233 68 1 527 172 1206 416 174 143 96 843 1...
96     156 87 673 479 153 91 519 8 484 645 64 100 170...
97     228 222 670 89 482 98 50 115 7 435 28 428 655 ...
98     47 517 116 629 523 514 25 428 435 211 210 659 194
99     4 79 1016 403 50 742 181 182 597 410 168 204 5...
Name: movie_id, Length: 942, dtype: object

In [57]:
import apyori

association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.2, 
                                   min_confidence=0.3, 
                                   min_lift=2, 
                                   min_length=2)

In [60]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [61]:
asr_df

,from,to,confidence,support,lift
0,172,174,0.819113,0.254777,2.21725
1,173,174,0.762097,0.200637,2.06292
2,174,195,0.548851,0.20276,2.38257
3,174,204,0.543103,0.200637,2.17704
4,174,210,0.563218,0.208068,2.25767
5,174,79,0.591954,0.218684,2.1122
6,172,181 174,0.651877,0.20276,2.6583
7,172,50 174,0.78157,0.2431,2.51276
8,172,181 50,0.750853,0.233546,2.02666
9,174,50 98,0.577586,0.213376,2.12534
